In [ ]:
import os
os.environ["TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL"] = "1"

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import load_dataset
import evaluate

In [ ]:
ner_datasets = load_dataset("lansinuote/peoples-daily-ner")

In [ ]:
ner_datasets

In [ ]:
label_list = ner_datasets["train"].features["ner_tags"].feature.names

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
model = AutoModelForTokenClassification.from_pretrained("hfl/chinese-macbert-base", num_labels=len(label_list))

In [ ]:
print(ner_datasets["train"][0]["tokens"])
print(ner_datasets["train"][0]["ner_tags"])
print(len(ner_datasets["train"][0]["tokens"]))
print(len(ner_datasets["train"][0]["ner_tags"]))

In [ ]:
input = tokenizer(ner_datasets["train"][0]["tokens"], is_split_into_words=True)
input

In [ ]:
input.word_ids(0)

In [ ]:
res = tokenizer("interesting word")
res

In [ ]:
res.word_ids()

In [ ]:
def process_function(examples, tokenizer=tokenizer):
    labels = []
    inputs = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = inputs.word_ids(batch_index=i)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    inputs["labels"] = labels
    return inputs

# 新的测试函数：只在子词处理上有差异
def process_function_test(examples, tokenizer=tokenizer):
    labels = []
    inputs = tokenizer(examples["tokens"], max_length=128, truncation=True, is_split_into_words=True)
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None  # 添加这个变量
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != previous_word_idx:  # 只有这里不同
                label_ids.append(label[word_id])
            else:
                label_ids.append(-100)  # 子词标签设为-100
            previous_word_idx = word_id  # 更新previous_word_idx
        labels.append(label_ids)
    inputs["labels"] = labels
    return inputs


ner_datasets["train"].select(range(2)).map(process_function, batched=True, remove_columns=ner_datasets["train"].column_names)

In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
def eval_matric(pred):
    predictions, labels = pred
    predictions = predictions.argmax(axis=-1)

    real_predictions = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    real_labels = [
        [label_list[l] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=real_predictions, references=real_labels, mode="struct", scheme="IOB2")

    flattened_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                flattened_results[f"{key}_{sub_key}"] = sub_value
        else:
            flattened_results[key] = value

    return flattened_results

In [ ]:
train_args = TrainingArguments(
    output_dir="./checkpoints-for-nre",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    logging_steps=10,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    num_train_epochs=3,
    save_strategy="epoch",
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    # logging_first_step=True,
    # log_level="info",
    # warmup_ratio=0.1,  # 添加warmup
    # weight_decay=0.01  # 添加权重衰减
)
train_args

In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=ner_datasets["train"].map(process_function_test, batched=True, remove_columns=ner_datasets["train"].column_names),
    eval_dataset=ner_datasets["validation"].map(process_function_test, batched=True, remove_columns=ner_datasets["validation"].column_names),
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
    compute_metrics=eval_matric
)

In [ ]:
trainer.train()

In [ ]:
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config

In [ ]:
from transformers import pipeline

pipe = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [ ]:
print(pipe("小明在北京"))
print(pipe("小明在北京工作"))
print(pipe("小明在北京上班"))
print(pipe("邓小平在北京"))
print(pipe("邓小平在北京上班"))
print(pipe("小渊惠三访问中国"))

In [ ]:
import torch

# 对比分析："小明在北京上班" vs "邓小平在北京上班"
test_pairs = [
    ("小明在北京上班", "小明"),
    ("邓小平在北京上班", "邓小平"),
]

device = next(model.parameters()).device

for text, name in test_pairs:
    print(f"\n{'='*80}")
    print(f"测试句子: {text}")
    print(f"目标人名: {name}")
    print('-'*80)

    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_labels = predictions[0].argmax(dim=-1)
    scores = predictions[0].max(dim=-1).values

    # 显示每个token的预测
    for i, (token, label_id, score) in enumerate(zip(tokens, predicted_labels, scores)):
        if token in ['[CLS]', '[SEP]']:
            continue

        label = label_list[label_id.item()]

        # 高亮显示人名token
        if token in name:
            marker = "👉" if label in ['B-PER', 'I-PER'] else "❌"
            print(f"{marker} {token:8s} -> {label:8s} (置信度: {score.item():.4f})")
        else:
            print(f"   {token:8s} -> {label:8s} (置信度: {score.item():.4f})")
